In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os

#随机初始化函数
def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / tf.sqrt(in_dim / 2.)
    return tf.random_normal(shape=size, stddev=xavier_stddev)


mb_size = 32
Z_dim = 16
c_dim = 10

X = tf.placeholder(tf.float32, shape=[None, 784])

#判别器中间参数
D_W1 = tf.Variable(xavier_init([784, 128]))
D_b1 = tf.Variable(tf.zeros(shape=[128]))

D_W2 = tf.Variable(xavier_init([128, 1]))
D_b2 = tf.Variable(tf.zeros(shape=[1]))

theta_D = [D_W1, D_W2, D_b1, D_b2]


#噪声
Z = tf.placeholder(tf.float32, shape=[None, Z_dim])
#隐含因子latent code
c = tf.placeholder(tf.float32, shape=[None, c_dim])

#生成器中间参数
G_W1 = tf.Variable(xavier_init([Z_dim+c_dim, 256]))
G_b1 = tf.Variable(tf.zeros(shape=[256]))

G_W2 = tf.Variable(xavier_init([256, 784]))
G_b2 = tf.Variable(tf.zeros(shape=[784]))

theta_G = [G_W1, G_W2, G_b1, G_b2]

#Q参数
Q_W1 = tf.Variable(xavier_init([784, 128]))
Q_b1 = tf.Variable(tf.zeros(shape=[128]))

Q_W2 = tf.Variable(xavier_init([128, c_dim]))
Q_b2 = tf.Variable(tf.zeros(shape=[c_dim]))

theta_Q = [Q_W1, Q_W2, Q_b1, Q_b2]


def sample_Z(m, n):
    return np.random.uniform(-1., 1., size=[m, n])


def sample_c(m):
    return np.random.multinomial(1, c_dim*[1/c_dim], size=m)


def generator(z, c):
    inputs = tf.concat(axis=1, values=[z, c])
    G_h1 = tf.nn.relu(tf.matmul(inputs, G_W1) + G_b1)
    G_log_prob = tf.matmul(G_h1, G_W2) + G_b2
    G_prob = tf.nn.sigmoid(G_log_prob)

    return G_prob


def discriminator(x):
    D_h1 = tf.nn.relu(tf.matmul(x, D_W1) + D_b1)
    D_logit = tf.matmul(D_h1, D_W2) + D_b2
    D_prob = tf.nn.sigmoid(D_logit)

    return D_prob


def dc_discriminator(x):
    x = tf.reshape(x, shape=[-1, 28, 28, 1])
    conv1 = tf.nn.relu(tf.nn.conv2d(x, DC_D_W1, strides=[1, 2, 2, 1], padding='SAME') + DC_D_b1)
    conv2 = tf.nn.relu(tf.nn.conv2d(conv1, DC_D_W2, strides=[1, 2, 2, 1], padding='SAME') + DC_D_b2)
    conv2 = tf.reshape(conv2, shape=[-1, 7 * 7 * 1])
    h = tf.nn.relu(tf.matmul(conv2, DC_D_W3) + DC_D_b3)
    logit = tf.matmul(h, DC_D_W4) + DC_D_b4
    prob = tf.nn.sigmoid(logit)

    return prob

def Q(x):
    Q_h1 = tf.nn.relu(tf.matmul(x, Q_W1) + Q_b1)
    Q_prob = tf.nn.softmax(tf.matmul(Q_h1, Q_W2) + Q_b2)

    return Q_prob


def plot(samples):
    fig = plt.figure(figsize=(10, c_dim))
    gs = gridspec.GridSpec(c_dim, 10)
    gs.update(wspace=0.05, hspace=0.05)
    
    for a in range(c_dim):
        for i, sample in enumerate(samples[a]):
            ax = plt.subplot(gs[i+a*c_dim])
            plt.axis('off')
            ax.set_xticklabels([])
            ax.set_yticklabels([])
            ax.set_aspect('equal')
            plt.imshow(sample.reshape(28, 28), cmap='Greys_r')
    
    return fig

G_sample = generator(Z, c)
D_real = discriminator(X)
D_fake = discriminator(G_sample)
Q_c_given_x = Q(G_sample)

D_loss = -tf.reduce_mean(tf.log(D_real + 1e-8) + tf.log(1 - D_fake + 1e-8))
G_loss = -tf.reduce_mean(tf.log(D_fake + 1e-8))

cross_ent = tf.reduce_mean(-tf.reduce_sum(tf.log(Q_c_given_x + 1e-8) * c, 1))
Q_loss = cross_ent

D_solver = tf.train.AdamOptimizer().minimize(D_loss, var_list=theta_D)
G_solver = tf.train.AdamOptimizer().minimize(G_loss, var_list=theta_G)
Q_solver = tf.train.AdamOptimizer().minimize(Q_loss, var_list=theta_Q + theta_G)

mnist = input_data.read_data_sets('./MNIST', one_hot=True)

sess = tf.Session()
summary_writer = tf.summary.FileWriter("infogan_logs/logs10/", sess.graph)
sess.run(tf.global_variables_initializer())

if not os.path.exists('dcout10/'):
    os.makedirs('dcout10/')

i = 0

for it in range(1000000):
    #每1000步输出一张图片
    if it % 1000 == 0:
        Z_noise = sample_Z(10, Z_dim)

        idx = np.random.randint(0, c_dim) #0-11的整数
#         c_noise = np.zeros([10, 12])
#         c_noise[range(10), 0] = 1   #令c_noise某一列为1
        
#         samples = sess.run(G_sample,
#                            feed_dict={Z: Z_noise, c: c_noise})
        sample_list = []
        for n in range(c_dim):
            c_noise = np.zeros([10,c_dim])
            c_noise[range(10),n] = 1
            samples = sess.run(G_sample,
                              feed_dict={Z: Z_noise, c: c_noise})
            sample_list.append(samples)

        #Tensorboard可视化
        with tf.name_scope('fake_image'):
            image_shaped = tf.reshape(samples[0],[-1,28,28,1])
            tf.summary.image('fake_image',image_shaped,10)
        merged = tf.summary.merge_all()
        summary = sess.run(merged)
        summary_writer.add_summary(summary,it)
        
        fig = plot(sample_list)
        plt.savefig('dcout10/{}.png'.format(str(i).zfill(3)), bbox_inches='tight')
        i += 1
        plt.close(fig)

    #训练参数
    X_mb, _ = mnist.train.next_batch(mb_size)
    Z_noise = sample_Z(mb_size, Z_dim)
    c_noise = sample_c(mb_size)

    _, D_loss_curr = sess.run([D_solver, D_loss],
                              feed_dict={X: X_mb, Z: Z_noise, c: c_noise})

    _, G_loss_curr = sess.run([G_solver, G_loss],
                              feed_dict={Z: Z_noise, c: c_noise})

    sess.run([Q_solver], feed_dict={Z: Z_noise, c: c_noise})

    if it % 100 == 0:
        print('Iter: {}'.format(it))
        print('D loss: {:.4}'. format(D_loss_curr))
        print('G_loss: {:.4}'.format(G_loss_curr))
        print()

In [ ]:
idx = np.random.randint(0, 10) #0-9的整数
c_noise = np.zeros([16, 10])
c_noise[range(16), idx] = 1
c_noise[range(16), idx+1] = 1
c_noise

In [ ]:
list(enumerate(c_noise))

In [ ]:
c_noise[:,0]

In [ ]:
import numpy as np
np.random.randint(0, 10)

投掷骰子，在12个位置中扔1次，每个位置中扔到骰子的次数。这里起到位置随机取1的作用。

In [ ]:
np.random.multinomial(1, 12*[1/12], size=mb_size)